In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession as ss

In [4]:
sp= ss.builder.appName('ass2').getOrCreate()

In [5]:
df=sp.read.csv('D:\\course material\\titanic_data_set\\train.csv', header=True, inferSchema=True)

In [6]:
df.select('Fare').show()

+-------+
|   Fare|
+-------+
|   7.25|
|71.2833|
|  7.925|
|   53.1|
|   8.05|
| 8.4583|
|51.8625|
| 21.075|
|11.1333|
|30.0708|
|   16.7|
|  26.55|
|   8.05|
| 31.275|
| 7.8542|
|   16.0|
| 29.125|
|   13.0|
|   18.0|
|  7.225|
+-------+
only showing top 20 rows



In [7]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
avg=df.describe('Age').show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [9]:
df.select('Age').show()

+----+
| Age|
+----+
|22.0|
|38.0|
|26.0|
|35.0|
|35.0|
|null|
|54.0|
| 2.0|
|27.0|
|14.0|
| 4.0|
|58.0|
|20.0|
|39.0|
|14.0|
|55.0|
| 2.0|
|null|
|31.0|
|null|
+----+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import avg
avg=df.select(avg('Age')).first()[0]

df=df.na.fill(value=avg, subset= 'Age')

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(
inputCols=['PassengerId','Pclass','Age','SibSp','Parch','Fare'],
    outputCol='features'
)

In [13]:
output=assembler.transform(df)

In [14]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[1.0,3.0,22.0,1.0...|
|[2.0,1.0,38.0,1.0...|
|[3.0,3.0,26.0,0.0...|
|[4.0,1.0,35.0,1.0...|
|[5.0,3.0,35.0,0.0...|
|[6.0,3.0,29.69911...|
|[7.0,1.0,54.0,0.0...|
|[8.0,3.0,2.0,3.0,...|
|[9.0,3.0,27.0,0.0...|
|[10.0,2.0,14.0,1....|
|[11.0,3.0,4.0,1.0...|
|[12.0,1.0,58.0,0....|
|[13.0,3.0,20.0,0....|
|[14.0,3.0,39.0,1....|
|[15.0,3.0,14.0,0....|
|[16.0,2.0,55.0,0....|
|[17.0,3.0,2.0,4.0...|
|[18.0,2.0,29.6991...|
|[19.0,3.0,31.0,1....|
|[20.0,3.0,29.6991...|
+--------------------+
only showing top 20 rows



In [15]:
fdata= output.select(['features', 'Survived'])

In [16]:
fdata.show()

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[1.0,3.0,22.0,1.0...|       0|
|[2.0,1.0,38.0,1.0...|       1|
|[3.0,3.0,26.0,0.0...|       1|
|[4.0,1.0,35.0,1.0...|       1|
|[5.0,3.0,35.0,0.0...|       0|
|[6.0,3.0,29.69911...|       0|
|[7.0,1.0,54.0,0.0...|       0|
|[8.0,3.0,2.0,3.0,...|       0|
|[9.0,3.0,27.0,0.0...|       1|
|[10.0,2.0,14.0,1....|       1|
|[11.0,3.0,4.0,1.0...|       1|
|[12.0,1.0,58.0,0....|       1|
|[13.0,3.0,20.0,0....|       0|
|[14.0,3.0,39.0,1....|       0|
|[15.0,3.0,14.0,0....|       0|
|[16.0,2.0,55.0,0....|       1|
|[17.0,3.0,2.0,4.0...|       0|
|[18.0,2.0,29.6991...|       1|
|[19.0,3.0,31.0,1....|       0|
|[20.0,3.0,29.6991...|       1|
+--------------------+--------+
only showing top 20 rows



In [17]:
test, train= fdata.randomSplit([0.25,0.75])

In [18]:
test

DataFrame[features: vector, Survived: int]

In [19]:
test.count()

201

In [20]:
train.count()

690

In [21]:
from pyspark.ml.classification import LogisticRegression

In [24]:
logReg=LogisticRegression(labelCol='Survived')

In [27]:
model=logReg.fit(train)

In [28]:
model_summary=model.summary

In [30]:
model_summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|          Survived|         prediction|
+-------+------------------+-------------------+
|  count|               690|                690|
|   mean|0.3884057971014493|0.26666666666666666|
| stddev|0.4877412266513293|0.44253743428668385|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [31]:
from pyspark.ml.evaluation import  BinaryClassificationEvaluator

In [32]:
preds_lbls=model.evaluate(test)

In [33]:
preds_lbls.predictions.show()

+--------------------+--------+--------------------+--------------------+----------+
|            features|Survived|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[3.0,3.0,26.0,0.0...|       1|[1.06843188286673...|[0.74429858894498...|       0.0|
|[7.0,1.0,54.0,0.0...|       0|[-0.0495946427618...|[0.48760388002630...|       1.0|
|[12.0,1.0,58.0,0....|       1|[0.24512588492926...|[0.56097645373870...|       0.0|
|[13.0,3.0,20.0,0....|       0|[0.83835210075483...|[0.69811803590810...|       0.0|
|[15.0,3.0,14.0,0....|       0|[0.60868627398166...|[0.64764106520364...|       0.0|
|[18.0,2.0,29.6991...|       1|[0.20820514564676...|[0.55186406536705...|       0.0|
|[19.0,3.0,31.0,1....|       0|[1.45086479919231...|[0.81013149174077...|       0.0|
|[21.0,2.0,35.0,0....|       0|[0.34106941403301...|[0.58445027297854...|       0.0|
|[24.0,1.0,28.0,0....|       1|[-0.9577067040171...|[0.2773375857

In [34]:
evalr=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [35]:
evalr.evaluate(preds_lbls.predictions)

0.6509895722494148